In [1]:
import json
import os
import shutil
from tqdm import tqdm
import pandas as pd
import random

# Configura la semilla para reproducibilidad
random.seed(42)

# Carga los datos del CSV generado
df = pd.read_csv("datos_ines.csv")

In [2]:
# Verificamos los primeros registros para entender la estructura
print("Estructura del DataFrame:")
df.head()

Estructura del DataFrame:


,Archivo_Imagen,Nombre,Segundo Nombre,Apellido Paterno,Apellido Materno,Estado,Municipio,Colonia,Calle,Numero Ext.,Numero Int.,Codigo Postal,CURP,Clave Elector,Fecha de Nacimiento,Seccion,Vigencia Ano Inicio,Vigencia Ano Final,Ano Registro,Mes Registro
0,ine_falsa_001.jpg,MARIA,CRISTINA,CABALLERO,MERINO,JAL,VIEJA ISLAS MARSHALL,DE LA MONTANA NORTE,PROLONGACION COLOMBIA,S/N,NaN,22469,NZMC668219HANLOWU3,GYOJVG646605138YLP,16/02/1936,7751,2020,2030,2006,12
1,ine_falsa_002.jpg,JORGE,NaN,CORTES,LUCIO,MIC,VIEJA CÔTE D'IVOIRE,LOS BAJOS NORTE,CALLEJON VERACRUZ DE IGNACIO DE LA LLAVE MZ 5 LT,78,PISO 16,86080,FKVT018422HLZFRFT5,JYZRXM75539847S0YM,07/09/1935,6114,2021,2031,2013,4
2,ine_falsa_003.jpg,SERGIO,NaN,PELAYO,REGALADO,VER,VIEJA EGIPTO,LOS BAJOS SUR,PROLONGACION JALISCO,8672F,NaN,79474,PKWI178127HGYEFS39,MSVBLJ76123588T2HV,16/08/1989,7959,2020,2030,2023,5
3,ine_falsa_004.jpg,EVA,NaN,HEREDIA,CASAS,CDMX,SAN ERIC LOS BAJOS,LOS BAJOS NORTE,PERIFERICO CEPEDA KM,11.8,NaN,30229,KMHY152680MERVSR95,KAWXDT233011761AWP,21/01/2006,3562,2029,2039,2009,9
4,ine_falsa_005.jpg,CORNELIO,NaN,MORENO,ROJAS,TLAX,SAN AIDA DE LA MONTANA,LOS ALTOS NORTE,ANDADOR RODRIGEZ,8256D,PISO 4,27362,AYYC944500HRJHVSZ6,HJCETD400179338AZ3,18/11/2003,6598,2025,2035,2009,8


In [3]:
print("\nColumnas disponibles:")
df.columns.tolist()


Columnas disponibles:


['Archivo_Imagen',
 'Nombre',
 'Segundo Nombre',
 'Apellido Paterno',
 'Apellido Materno',
 'Estado',
 'Municipio',
 'Colonia',
 'Calle',
 'Numero Ext.',
 'Numero Int.',
 'Codigo Postal',
 'CURP',
 'Clave Elector',
 'Fecha de Nacimiento',
 'Seccion',
 'Vigencia Ano Inicio',
 'Vigencia Ano Final',
 'Ano Registro',
 'Mes Registro']

In [4]:
def create_sets(df, source_folder, train=0.7, val=0.2, test=0.1):
    """
    Crea los conjuntos de entrenamiento, validación y prueba para el modelo Donut.
    
    Args:
        df: DataFrame con los datos de las INEs
        source_folder: Carpeta donde se encuentran las imágenes originales
        train, val, test: Proporciones para cada conjunto
    """
    if abs((train + val + test) - 1.0) > 0.001:  # Permite pequeñas diferencias de redondeo
        raise ValueError(f"Las proporciones deben sumar 1.0, pero suman {train + val + test}")

    # Crear la estructura de carpetas
    # Carpeta raíz donde están las imágenes originales
    base_folder = source_folder
    
    # Subcarpeta 'data' para todas las INEs
    data_folder = os.path.join(base_folder, "data")
    os.makedirs(data_folder, exist_ok=True)
    
    # Subcarpeta 'sets' para train/val/test
    sets_folder = os.path.join(base_folder, "sets")
    os.makedirs(sets_folder, exist_ok=True)
    
    # Carpetas para cada conjunto en 'sets'
    train_folder = os.path.join(sets_folder, "train")
    val_folder = os.path.join(sets_folder, "val")
    test_folder = os.path.join(sets_folder, "test")
    
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)
    
    # Eliminar los archivos metadata.jsonl existentes si los hay
    for folder in [train_folder, val_folder, test_folder]:
        metadata_path = os.path.join(folder, "metadata.jsonl")
        if os.path.exists(metadata_path):
            os.remove(metadata_path)

    # Mezclar los datos aleatoriamente
    samples = df.sample(frac=1.0).reset_index(drop=True)
    
    # Calcular el número de imágenes para cada conjunto
    n = len(samples)
    n_train = int(train * n)
    n_val = int(val * n)
    # El resto va para test para asegurar que no se pierdan muestras por redondeo
    n_test = n - n_train - n_val
    
    print(f"Total de muestras: {n}")
    print(f"Muestras para entrenamiento: {n_train}")
    print(f"Muestras para validación: {n_val}")
    print(f"Muestras para prueba: {n_test}")
    
    # Primero, copiar todas las imágenes a la carpeta 'data'
    print("Copiando todas las imágenes a la carpeta 'data'...")
    for idx, row in tqdm(samples.iterrows(), total=samples.shape[0], desc="Copiando imágenes originales"):
        file_name = row["Archivo_Imagen"]
        image_path = os.path.join(source_folder, file_name)
        dest_path = os.path.join(data_folder, file_name)
        
        # Solo copiar si no existe ya
        if not os.path.exists(dest_path):
            shutil.copyfile(image_path, dest_path)
    
    # Ahora, crear los conjuntos en la carpeta 'sets'
    for idx, row in tqdm(samples.iterrows(), total=samples.shape[0], desc="Creando conjuntos"):
        # Obtener el nombre del archivo de imagen
        file_name = row["Archivo_Imagen"]
        
        # Crear diccionario con los datos de la INE para el ground truth
        data = {
            "nombre": row["Nombre"],
            "segundo_nombre": row["Segundo Nombre"],
            "apellido_paterno": row["Apellido Paterno"],
            "apellido_materno": row["Apellido Materno"],
            "estado": row["Estado"],
            "municipio": row["Municipio"],
            "colonia": row["Colonia"],
            "calle": row["Calle"],
            "numero_ext": row["Numero Ext."],
            "numero_int": row["Numero Int."],
            "codigo_postal": row["Codigo Postal"],
            "curp": row["CURP"],
            "clave_elector": row["Clave Elector"],
            "fecha_nacimiento": row["Fecha de Nacimiento"],
            "seccion": row["Seccion"],
            "vigencia_inicio": row["Vigencia Ano Inicio"],
            "vigencia_fin": row["Vigencia Ano Final"],
            "ano_registro": row["Ano Registro"],
            "mes_registro": row["Mes Registro"]
        }
        
        # Crear el formato esperado por Donut
        gt_parse = {"gt_parse": data}
        
        line = {
            "file_name": file_name,
            "ground_truth": json.dumps(gt_parse)
        }
        
        # Ruta de la imagen en la carpeta 'data'
        image_path = os.path.join(data_folder, file_name)
        
        # Copiar la imagen y añadir la línea a metadata.jsonl según el conjunto
        if idx < n_train:
            dest_path = os.path.join(train_folder, file_name)
            shutil.copyfile(image_path, dest_path)
            with open(os.path.join(train_folder, "metadata.jsonl"), "a") as f:
                f.write(json.dumps(line) + "\n")
        
        elif idx < n_train + n_val:
            dest_path = os.path.join(val_folder, file_name)
            shutil.copyfile(image_path, dest_path)
            with open(os.path.join(val_folder, "metadata.jsonl"), "a") as f:
                f.write(json.dumps(line) + "\n")
        
        else:
            dest_path = os.path.join(test_folder, file_name)
            shutil.copyfile(image_path, dest_path)
            with open(os.path.join(test_folder, "metadata.jsonl"), "a") as f:
                f.write(json.dumps(line) + "\n")
    
    print("Estructura de carpetas y conjuntos creados exitosamente.")
    return data_folder, train_folder, val_folder, test_folder

In [5]:
# Ruta donde se encuentran las imágenes generadas
CARPETA_IMAGENES = r"C:\Users\Adrian\Desktop\INEs Dataset"  # Ajusta esta ruta según tu configuración

# Crear los conjuntos de entrenamiento, validación y prueba
data_folder, train_folder, val_folder, test_folder = create_sets(df, CARPETA_IMAGENES)

Total de muestras: 300
Muestras para entrenamiento: 210
Muestras para validación: 60
Muestras para prueba: 30
Copiando todas las imágenes a la carpeta 'data'...


Creando conjuntos: 100%|██████████| 300/300 [00:00<00:00, 1026.46it/s]

Estructura de carpetas y conjuntos creados exitosamente.
